In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from tqdm import tqdm

import torch.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch

import matplotlib.pyplot as plt
import datetime
import pickle
import gzip
import os

np.random.seed(datetime.datetime.now().microsecond)

PEAK_THRESHOLD = 390
DATA_PATH = "./data/exported/30sec/"

# Data Load

## Sitting

In [ ]:
# Sitting data load
with gzip.open(DATA_PATH + "sitting_ecg.pkl", "rb") as f:
    sitting_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "sitting_acc.pkl", "rb") as f:
    sitting_acc = pickle.load(f)
sitting_acc = sitting_acc - np.mean(sitting_acc)

In [ ]:
fig, ax = plt.subplots(len(sitting_acc), 1, figsize=(20, 10))
for idx in range(len(sitting_acc)):
    peaks, _ = find_peaks(sitting_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(sitting_acc[idx])
    ax[idx].plot(peaks, sitting_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(sitting_acc)):
    peaks, _ = find_peaks(sitting_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_sitting = sitting_ecg
y_sitting = peak_list

## Walking

In [ ]:
# Walking data load
with gzip.open(DATA_PATH + "walking_ecg.pkl", "rb") as f:
    walking_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "walking_acc.pkl", "rb") as f:
    walking_acc = pickle.load(f)
walking_acc = walking_acc - np.mean(walking_acc)

In [ ]:
fig, ax = plt.subplots(len(walking_acc), 1, figsize=(20, 20))
for idx in range(len(walking_acc)):
    peaks, _ = find_peaks(walking_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(walking_acc[idx])
    ax[idx].plot(peaks, walking_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(walking_acc)):
    peaks, _ = find_peaks(walking_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_walking = walking_ecg
y_walking = peak_list

In [ ]:
print(y_walking)

## Running

In [ ]:
# Running data load
with gzip.open(DATA_PATH + "running_ecg.pkl", "rb") as f:
    running_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "running_acc.pkl", "rb") as f:
    running_acc = pickle.load(f)
running_acc = running_acc - np.mean(running_acc)

In [ ]:
fig, ax = plt.subplots(len(running_acc), 1, figsize=(20, 10))
for idx in range(len(running_acc)):
    peaks, _ = find_peaks(running_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(running_acc[idx])
    ax[idx].plot(peaks, running_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(running_acc)):
    peaks, _ = find_peaks(running_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_running = running_ecg
y_running = peak_list

# X, y data Split

In [ ]:
BATCH_SIZE = 1

In [ ]:
X = np.concatenate((X_walking, X_running, X_sitting))
y = np.concatenate((y_walking, y_running, y_sitting))

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# y = scaler.fit_transform(y.reshape(-1, 1))

print(f"""X shape: {X.shape}
y shape: {y.shape}""")

In [ ]:
plotting_idx = np.random.randint(0, len(X), 3)

# Plotting the data
fig, ax = plt.subplots(3, 1, figsize=(20, 10))

for idx in range(3):
    ax[idx].plot(X[plotting_idx[idx]])
    ax[idx].set_title(f"Peak count: {y[plotting_idx[idx]]}")
plt.show()


In [ ]:
with gzip.open('./data//exported/X.pkl', 'wb') as f:
    pickle.dump(X, f)

with gzip.open('./data//exported/y.pkl', 'wb') as f:
    pickle.dump(y, f)

In [ ]:
with gzip.open('./data//exported/X.pkl', 'rb') as f:
    X = pickle.load(f)

with gzip.open('./data//exported/y.pkl', 'rb') as f:
    y = pickle.load(f)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=datetime.datetime.now().microsecond)

print(f"""X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape: {X_test.shape}
y_test shape: {y_test.shape}""")

In [ ]:
class ECGDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = np.array(X).astype(np.float32)
        self.y = np.array(y).astype(np.float32)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]

        return X, y

    def __len__(self):
        return len(self.X)
    
train_dataset = ECGDataset(X_train, y_train)
test_dataset = ECGDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model

In [ ]:
device = torch.device("mps")
EPOCH = 50

In [ ]:
class PositionEmbedding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionEmbedding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len

        self.position_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        positions = torch.arange(0, x.size(1)).unsqueeze(0).repeat(x.size(0), 1).to(device)
        return self.position_embedding(positions)
    
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.dim_feedforward = dim_feedforward
        self.num_layers = num_layers
        self.dropout = dropout

        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    def forward(self, x, mask=None):
        return self.transformer_encoder(x, mask)
    
class InputConvolutions(nn.Module):
    def __init__(self, d_model, kernel_size, stride, padding):
        super(InputConvolutions, self).__init__()
        self.d_model = d_model
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.conv = nn.Conv1d(1, d_model, kernel_size, stride, padding)
        self.conv1 = nn.Conv1d(d_model, d_model, 1, stride, padding)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.conv1(x)
        x = self.relu(x)
        return x
    
class RegressionModel(nn.Module):
    def __init__(self, d_model):
        super(RegressionModel, self).__init__()
        self.d_model = d_model

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(16384, d_model * 4)
        self.linear3 = nn.Linear(d_model * 4, d_model)
        self.linear4 = nn.Linear(d_model, 1)
        self.linear5 = nn.Linear(1, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.relu(x)

        x = self.linear3(x)
        x = self.relu(x)

        x = self.linear4(x)
        x = torch.squeeze(x)
        
        return x
    
class RNN_Model(nn.Module):
    def __init__(self, d_model, n_head):
        super(RNN_Model, self).__init__()
        self.d_model = d_model

        self.rnn = nn.GRU(d_model, d_model, n_head, batch_first=True)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.rnn(x)
        return x
    
class Model(nn.Module):
    def __init__(self, d_model: int, nhead: int, dim_feedforward: int, num_layers: int, dropout: float = 0.1):
        super(Model, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.dim_feedforward = dim_feedforward
        self.num_layers = num_layers
        self.dropout = dropout

        self.input_convolutions = InputConvolutions(d_model, 3, 1, 1)
        self.position_embedding = PositionEmbedding(d_model)
        self.transformer_encoder = TransformerEncoder(d_model, nhead, dim_feedforward, num_layers, dropout)
        self.rnn_model = RNN_Model(d_model, nhead)
        self.linear_model = RegressionModel(d_model)

    def forward(self, x):
        x = self.input_convolutions(x)
        x = self.position_embedding(x)
        x = self.transformer_encoder(x)
        x = self.rnn_model(x)
        x = self.linear_model(x)
        return x

In [ ]:
model = Model(
    d_model=128,
    nhead=8,
    dim_feedforward=256,
    num_layers=4,
    dropout=0.1
).to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(model)

# Model Train

In [ ]:
model.train()

train_loss, val_loss = [], []
for epoch in range(EPOCH):
    running_loss = 0.0
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs, labels = data
        inputs = inputs.reshape(-1, 1, 7500).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss.append(running_loss / len(train_dataloader))

    model.eval()
    with torch.no_grad():
        running_loss = 0.0
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

        val_loss.append(running_loss / len(test_dataloader))
    
    model.train()
    print(f"Epoch: {epoch + 1} / {EPOCH} | Train loss: {train_loss[-1]:.5f} | Val loss: {val_loss[-1]:.5f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Train loss")
plt.plot(val_loss, label="Val loss")
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        inputs = inputs.unsqueeze(1).to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        print(f"Loss: {loss:.5f}\t\tPredicted: {outputs.item()}\t\tActual: {labels.item():.5f}")